# OpenStack_WordCount

In [1]:
import pandas as pd
from nltk.corpus import stopwords 
import nltk
from nltk.corpus import treebank
import nltk.stem 
from nltk import stem
import numpy as np
import string
import re
import random
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer

In [2]:
#Randomly select 1000 from bugslist and report (launch pad and stack over flow)
from nltk.stem import PorterStemmer
ps = PorterStemmer()


In [3]:
filename = "buglist.xlsx"
df_launchpad=pd.read_excel(filename)
df=pd.DataFrame(df_launchpad)
del(df_launchpad)
df.set_index=df.bug
df=df.drop(['project','component','version','fault_class','fault_type','fault_symptom','severity','priority','status','mitigation','log','repro','submitter','assignee','created','deployment'], axis=1)

In [4]:
len(df)

17326

In [5]:

filename = "ask_os.csv"
df_ask_os = pd.read_csv(filename)
df_ask=pd.DataFrame(df_ask_os)
del(df_ask_os)
df_ask.set_index=df_ask.url
df_ask=df_ask.drop(['Serial','answer'],axis=1)


In [6]:
filename = "report.xlsx"
df_stackoverflow = pd.read_excel(filename)

df_st=pd.DataFrame(df_stackoverflow)
del(df_stackoverflow)
df_st.set_index=df_st.id
df_st=df_st.drop(['answers','link','Creator','date','votes','fetch_url'],axis=1)


In [7]:
stop_words=stopwords.words('english')
stop_word_dict=[]
for i in stop_words:
    stop_word_dict.append(str(i.encode("utf-8")))
common_words=['',"aren't",'ffffffffff']
stop_word_dict=stop_word_dict + common_words

In [8]:
def create_dict(dataframe,field,stop_word_dict):
    corpus=[]
    for i in field:
        
        if type(i)== int or type(i)== float:  
            continue
        i = re.sub(r'[^\x00-\x7f]','',i)
        for k in i:
            if type(k) == type(u''):
                k = k.encode("utf-8",'ignore')
        corpus.append(str(i).strip())
    dataframe['field_corpus']=pd.Series(corpus)
    dataframe['field_corpus']=pd.Series( str(i).lower()  if len(str(i)) >0 else '' for i in dataframe.field_corpus)
    corpus=[]
    for i in dataframe.field_corpus:
        data=(str(i))
        data=re.sub(r'[^\x00-\x7f]','',data) or re.sub(r'[0-9]','',data)
        corpus.append(data) 
    dataframe.field_corpus_new=pd.Series(corpus)
    corpus=[]
    new_corpus=[]
    row_corpus=[]
    for i in dataframe.field_corpus_new:
        substr=re.split('[^A-Za-z]',i)
        for j in substr:
            j=j.strip()
            if j not in stop_word_dict and len(j)>1:
                corpus.append(ps.stem(j))   
        new_corpus.append(corpus)
        corpus=[]
    dataframe.field_words_bag=pd.Series(new_corpus)
    field_words_count={}
    for i in new_corpus:
        for w in i:
            field_words_count[w]=field_words_count.get(w,0)+1 
    return field_words_count,dataframe.field_words_bag

In [9]:
my_words_count={}

In [10]:

Fault_Details_lp= pd.Series(df.fault_description+df.req)
#df=df.drop(['fault_description','req'],axis=1)
df=df.assign(Fault_Details_lp=Fault_Details_lp.values)
launchpad_words_count={}
launchpad_words_count,df['field_words_bag']=create_dict(df,df.Fault_Details_lp,stop_word_dict)
#df=df.drop(['Fault_Details_lp'],axis=1)
for k,v in launchpad_words_count.items():
    my_words_count[k]=my_words_count.get(k,1)+v
del(launchpad_words_count)


In [11]:
len(my_words_count)

31659

In [12]:

Fault_Details_st= pd.Series(df_st.title+df_st.tags+ df_st.longtext.to_string())
#df_st=df_st.drop(['tags','title','longtext'],axis=1)
df_st=df_st.assign(Fault_Details_st=Fault_Details_st.values)
stackoverflow_words_count={}
stackoverflow_words_count,df_st['field_words_bag']=create_dict(df_st,df_st.Fault_Details_st,stop_word_dict)
#df_st=df_st.drop(['Fault_Details_st'],axis=1)
for k,v in stackoverflow_words_count.items():
    my_words_count[k]=my_words_count.get(k,1)+v
del(stackoverflow_words_count)



In [13]:
len(my_words_count)

32031

In [14]:
Fault_Details_ask = pd.Series(df_ask.summary+df_ask.title)
#df_ask=df_ask.drop(['summary','title'],axis=1)
df_ask=df_ask.assign(Fault_Details_ask=Fault_Details_ask.values)
ask_words_count={}
ask_words_count,df_ask['field_words_bag']=create_dict(df_ask,df_ask.Fault_Details_ask,stop_word_dict)
#df_ask=df_ask.drop(['Fault_Details_ask'],axis=1)
for k,v in ask_words_count.items():
    my_words_count[k]=my_words_count.get(k,1)+v
del(ask_words_count)

In [15]:
df_ask.head()

,summary,title,url,Fault_Details_ask,field_corpus,field_words_bag
0,<p>Hi \nam trying to install Magnum-api and ma...,Unable to locate package Magnum-api,https://ask.openstack.org/en/question/103555/u...,<p>Hi \nam trying to install Magnum-api and ma...,<p>hi \nam trying to install magnum-api and ma...,"[hi, tri, instal, magnum, api, magnum, conduct..."
1,<p>Hello\nbased with these documentation\nhttp...,detailed documentation how to install Mitaka w...,https://ask.openstack.org/en/question/103553/d...,<p>Hello\nbased with these documentation\nhttp...,<p>hello\nbased with these documentation\nhttp...,"[hello, base, document, http, doc, openstack, ..."
2,"<div class=""snippet""><p>These controller nodes...",Galera breaks then all the controllers go down,https://ask.openstack.org/en/question/103510/g...,"<div class=""snippet""><p>These controller nodes...","<div class=""snippet""><p>these controller nodes...","[div, class, snippet, control, node, reboot, s..."
3,"<p>Hi, i'm now including new compute nodes to ...",Run kolla-ansible bootstrap-servers to specifi...,https://ask.openstack.org/en/question/103552/r...,"<p>Hi, i'm now including new compute nodes to ...","<p>hi, i'm now including new compute nodes to ...","[hi, includ, new, comput, node, exist, opensta..."
4,"<p>Hi all,</p>\n\n<p>I am new to OpenStack Hea...",How to use heat template to create MySQL serve...,https://ask.openstack.org/en/question/103384/h...,"<p>Hi all,</p>\n\n<p>I am new to OpenStack Hea...","<p>hi all,</p>\n\n<p>i am new to openstack hea...","[hi, new, openstack, heat, would, like, use, s..."


In [16]:
len(my_words_count)


57449

In [17]:
#Remove non-ascii character to process data using NLTK
#NLTK coverts string into meaningful words called token, removes punctuations and special characters.
#Analyzing Fault_desc and Req field from data set
#write process data to csv file

In [18]:
#launchpad_words_count={}
#launchpad_words_count,df['field_words_bag']=create_dict(df,df.Fault_Details_lp,stop_word_dict)

In [19]:
mylist=[]
for key, value in sorted(my_words_count.iteritems(), key=lambda (k,v): (v,k), reverse=True):
    mylist.append([key,value])
df_wordcount=pd.DataFrame(mylist, columns =['Key','Count'])
#del (mylist)

In [20]:
#del (my_words_count)


In [21]:
len(df_wordcount)

57449

In [22]:
df_wordcount.to_csv('dictionary.csv')

In [23]:
print "dictionary created"

dictionary created


In [24]:
mydict=set(df_wordcount.Key)

In [25]:
del (df_wordcount)

In [26]:
#Topic Modeling: This script loads a gensim dictionary and associated corpus and applies an LDA model.

<a href="http://radimrehurek.com/topic_modeling_tutorial/2%20-%20Topic%20Modeling.html"> Topic Modeling for Fun and Profit </a>

In [22]:
from gensim import corpora, models , similarities
import pyLDAvis.gensim
from optparse import OptionParser

C:\Users\zaina\Anaconda2\lib\site-packages\gensim\utils.py:855: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [23]:
pyLDAvis.enable_notebook()

In [ ]:
#build Dictinoary

In [30]:
opts= "open_stack"
dictionary = corpora.Dictionary([mydict])
dictionary.compactify()

In [31]:
print("Dictionary: ")
print(dictionary)

Dictionary: 
Dictionary(57679 unique tokens: [u'swdhome', u'htmlaccident', u'mjflmwmilcaichvibgljvvjmijogimh', u'pkmgksasi', u'ozfho']...)


In [32]:
#Save dictionary for future use

In [33]:
del(mydict)

In [34]:
dictionary.save("open_stack.dict")

In [35]:
#build Corpus

In [36]:
corpus =[dictionary.doc2bow(df.field_words_bag.loc[i]) for i in range(0,len(df))] 
corpus= corpus + [dictionary.doc2bow(df_st.field_words_bag.loc[i]) for i in range(0,len(df_st))] 
corpus= corpus +[dictionary.doc2bow(df_ask.field_words_bag.loc[i]) for i in range(0,len(df_ask))]

In [37]:
#Save corpus for future use

In [38]:
corpora.MmCorpus.serialize("open_stack.mm", corpus)

In [39]:
# Load the corpus and Dictionary

In [24]:
corpus = corpora.MmCorpus("open_stack.mm")
dictionary = corpora.Dictionary.load("open_stack.dict")

In [41]:
# Initialize Parameters

In [27]:
num_topics,passes,alpha=5,20,0.01

In [28]:
#Apply LDA


In [29]:
lda = models.LdaModel(corpus, id2word=dictionary,
                        num_topics=num_topics,
                        passes=passes,
                        alpha =alpha)

In [ ]:
#Save lda results for future reference

In [ ]:
lda.save("open_stack.lda")

In [30]:
lda=models.LdaModel.load('open_stack.lda',mmap='r')

In [31]:
top_words = [[word for word in lda.show_topic(topicno, topn=5)] for topicno in range(5)]
for i in top_words:
    print i

[(u'network', 0.020525661313409882), (u'code', 0.015209313647057266), (u'instanc', 0.012569376822856731), (u'node', 0.012553155741632153), (u'ip', 0.012090819308485282)]
[(u'openstack', 0.063726644864840504), (u'nova', 0.02449840633981323), (u'instal', 0.02430762270872705), (u'use', 0.023819859593658094), (u'tri', 0.022004846871574484)]
[(u'test', 0.019880705481180518), (u'openstack', 0.01851822476394208), (u'org', 0.018370467322106426), (u'https', 0.017524987717491385), (u'instal', 0.014228456575570857)]
[(u'nova', 0.058013282028327548), (u'py', 0.035673376152863152), (u'lib', 0.033724454272338639), (u'file', 0.032581795241183091), (u'usr', 0.032238469524396941)]
[(u'http', 0.020794664768736822), (u'keyston', 0.017271553209166034), (u'swift', 0.015510824756368601), (u'code', 0.015059128068668782), (u'auth', 0.012924926325462093)]


In [ ]:
# First LDA model with 5 topics, 20 passes, alpha = 0.001

In [ ]:
openstack_data =  pyLDAvis.gensim.prepare(lda, corpus, dictionary)
pyLDAvis.display(openstack_data)

In [ ]:
pyLDAvis.save_html(openstack_data,"openstack.html")
#pyLDAvis.save_html(vis_data,outpth+'LDA_Visualization.html')

In [32]:
print(lda)

LdaModel(num_terms=57679, num_topics=5, decay=0.5, chunksize=2000)


In [ ]:
token_table=pd.DataFrame(openstack_data.token_table )
token_table.to_csv("open_stack_token_table.csv")

In [42]:
topic=[]
for i in df.field_words_bag:
    bow_vector = dictionary.doc2bow(i)
    lda_vector = lda[bow_vector]
    lda_vector=sorted(lda_vector, key=lambda x: x[1],reverse=True)
    topic.append(lda_vector[0][-2])

In [43]:
df['Cluster']=pd.Series(topic)

In [44]:
df.to_csv('df.csv', encoding='utf-8')

In [45]:
del(df)

In [46]:
topic=[]
for i in df_st.field_words_bag:
    bow_vector = dictionary.doc2bow(i)
    lda_vector = lda[bow_vector]
    lda_vector=sorted(lda_vector, key=lambda x: x[1],reverse=True)
    topic.append(lda_vector[0][-2])

In [47]:
df_st['Cluster']=pd.Series(topic)

In [48]:
df_st.to_csv('df_st.csv', encoding='utf-8')


In [49]:
del(df_st)

In [50]:
topic=[]
for i in df_ask.field_words_bag:
    bow_vector = dictionary.doc2bow(i)
    lda_vector = lda[bow_vector]
    lda_vector=sorted(lda_vector, key=lambda x: x[1],reverse=True)
    topic.append(lda_vector[0][-2])

In [51]:
df_ask['Cluster']=pd.Series(topic)

In [52]:
df_ask.to_csv('df_ask.csv', encoding='utf-8')